In [17]:
import cv2
import numpy as np
import os
from tensorflow import keras
import time
import mediapipe as mp
import csv
import pandas as pd
import seaborn as sns
from matplotlib import patches

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Input, Dense, LSTM, Bidirectional
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
import time
import datetime
import timeit

from tensorflow import keras 
from PIL import ImageFont ,ImageDraw ,Image

%matplotlib inline

In [18]:
classes = ['자극', '당뇨병', '면역', '감기', '변비', '붕대', '설사', '성병', '소화제', '수면제',
       '회복', '입원', '진단서', '치료', '퇴원', '빈혈', '화상', '술', '커피', '의사', '간호사',
       '금식', '금연', '금주', '식도염', '숨차다', '통증', '가렵다', '답답', '건강', '불안',
       '검사', '팔', '아프다', '춥다', '머리', '충혈', '왼쪽', '오른쪽', '떨다', '몸', '전염',
       '병원', '병', '상처', '병원', '붓다', '피곤', '중독', '치매', '환자', '충격', '노화',
       '가루약', '물약', '약효', '무기력', '체온']

In [19]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [20]:
#랜드마크 그려주기
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) #POSE 랜드마크
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #왼손 랜드마크
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #오른손 랜드마크

In [21]:
##keypoint 좌표 반화
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([pose, lh, rh])

def extract_keypoints_pose(result):
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    return np.concatenate([pose])

def extract_keypoints_hand(result):
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    return np.concatenate([lh, rh])   

In [22]:
## mediapipe로 관절 포인트 표시
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image, results

In [23]:
sequence_lenght = 30

In [24]:
label_map = {label:num for num, label in enumerate(classes)}

In [25]:
label_map

{'자극': 0,
 '당뇨병': 1,
 '면역': 2,
 '감기': 3,
 '변비': 4,
 '붕대': 5,
 '설사': 6,
 '성병': 7,
 '소화제': 8,
 '수면제': 9,
 '회복': 10,
 '입원': 11,
 '진단서': 12,
 '치료': 13,
 '퇴원': 14,
 '빈혈': 15,
 '화상': 16,
 '술': 17,
 '커피': 18,
 '의사': 19,
 '간호사': 20,
 '금식': 21,
 '금연': 22,
 '금주': 23,
 '식도염': 24,
 '숨차다': 25,
 '통증': 26,
 '가렵다': 27,
 '답답': 28,
 '건강': 29,
 '불안': 30,
 '검사': 31,
 '팔': 32,
 '아프다': 33,
 '춥다': 34,
 '머리': 35,
 '충혈': 36,
 '왼쪽': 37,
 '오른쪽': 38,
 '떨다': 39,
 '몸': 40,
 '전염': 41,
 '병원': 45,
 '병': 43,
 '상처': 44,
 '붓다': 46,
 '피곤': 47,
 '중독': 48,
 '치매': 49,
 '환자': 50,
 '충격': 51,
 '노화': 52,
 '가루약': 53,
 '물약': 54,
 '약효': 55,
 '무기력': 56,
 '체온': 57}

In [26]:
new_model = tf.keras.models.load_model('./완성 모델/단어58개.h5')

In [28]:
sequence = []
sentence = []
threshold = 0.7
i = 2
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # feed 읽기
        ret, frame = cap.read()
        start_t = timeit.default_timer()
        img, results = mediapipe_detection(frame, holistic)
        
        #랜드마크 그리기
        draw_landmarks(img,results)
        
        
#       keypoints = extract_keypoints_pose(result)
        
        keypoint = extract_keypoints_pose(results)
        #예측
#         print(keypoint[65])
        if keypoint[65]<0.8 or keypoint[61]<0.8:
            keypoints = extract_keypoints(results)
            keypoints = np.concatenate([keypoints[0:99], keypoints[132:258]])
            sequence.insert(0,keypoints)
            sequence.insert(0,keypoints)
            sequence = sequence[:30]
            if i%30 == 0:
                res = new_model.predict(np.expand_dims(sequence, axis=0))[0]
                print(classes[np.argmax(res)])
                print(res[np.argmax(res)])
                if res[np.argmax(res)] > threshold:
                    sentence.append(classes[np.argmax(res)])
            
        else:
#             print(i)
            i=0
            sequence=[]
        if len(sentence) > 5:
            sentence = sentence[-5:]
        fontpath = "fonts/gulim.ttc"
        font = ImageFont.truetype(fontpath, 20)
        img_pil = Image.fromarray(img)
        draw = ImageDraw.Draw(img_pil)
        
        cv2.rectangle(img,(0,0),(640,40),(245,117,16),-1)
        #cv2.putText(img, ''.join(sentence),,
                   ##cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255,0),2,cv2.LINE_AA)
        draw.text((3,30),  ''.join(sentence), font=font, fill=(0,0,0,0))
        draw.text((450,30),  str(i), font=font, fill=(0,255,0,0))
        img = np.array(img_pil)
        terminate_t = timeit.default_timer()
        FPS = int(1./(terminate_t - start_t ))
        #화면에 보여주기
        cv2.imshow("OpenCV Feed",img)
        #화면 종료
        i+=2
        if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
    cap.release()
    cv2.destroyAllWindows()

오른쪽
0.039321445
오른쪽
0.039321445
오른쪽
0.9870036
팔
0.8186332
팔
0.820282
체온
0.89004254
팔
0.9196147
왼쪽
0.9965804
오른쪽
0.5313175
오른쪽
0.9831707
왼쪽
0.98972964
가렵다
0.76533467
왼쪽
0.65800595
왼쪽
0.50050074


In [12]:
np.shape(sequence)

(2, 225)

In [13]:
import numpy as np

In [14]:
data = np.arange(0,100)

In [15]:
data[100:130]

array([], dtype=int32)

In [16]:
data[30:]

array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
       47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
       64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80,
       81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
       98, 99])